# Overview

This project looks at airfare data covering the top 1,000 contiguous state city-pair markets. The challenge and dataset can be found [here](https://discuss.codecademy.com/t/data-science-independent-project-5-analyze-airfare-data/419949/1).

# Dataset

To download the dataset for this project, uncomment and run the next cell. Note it requires `wget`.

In [1]:
!wget -O airfare_data.csv https://static-assets.codecademy.com/community/datasets_forum_projects/airfare_data.csv

--2023-11-02 21:08:51--  https://static-assets.codecademy.com/community/datasets_forum_projects/airfare_data.csv
Resolving static-assets.codecademy.com (static-assets.codecademy.com)... 2606:4700::6811:d451, 2606:4700::6812:c73f, 104.17.212.81, ...
Connecting to static-assets.codecademy.com (static-assets.codecademy.com)|2606:4700::6811:d451|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18217054 (17M) [text/csv]
Saving to: ‘airfare_data.csv’

airfare_data.csv    100%[===================>]  17.37M  18.3MB/s    in 0.9s    

2023-11-02 21:08:53 (18.3 MB/s) - ‘airfare_data.csv’ saved [18217054/18217054]



In [1]:
import pandas as pd
import sqlite3

data = pd.read_csv('airfare_data.csv')
data.head(1)

,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,nsmiles,passengers,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,table_1_flag,Geocoded_City1,Geocoded_City2
0,2009,2,32467,34576,"Miami, FL (Metropolitan Area)","Rochester, NY",1204,203,151.46,FL,0.29,131.05,FL,0.29,131.05,1,"Miami, FL (Metropolitan Area)\n(44.977479, -93...","Rochester, NY\n(43.155708, -77.612547)"


In [3]:

conn = sqlite3.connect("airfare.db")
cur = conn.cursor()

cur.execute('''CREATE TABLE airfare (
            year INT
            quarter INT
            citymarketid_1 INT
            citymarketid_2 INT
            city1 TEXT
            city2 TEXT
            nsmiles INT
            passengers INT
            fare REAL
            carrier_lg TEXT
            large_ms REAL
            fare_lg REAL
            carrier_low TEXT
            lf_ms REAL
            fare_low REAL
            table_1_flag_INT
            geocoded_city1 TEXT
            geocoded_city2 TEXT)''')

In [4]:
data.to_sql('airfare', conn, if_exists='replace', index=False)

90021

## Exploration

In [2]:
# Load the sql module to iPython

%load_ext sql

In [3]:
%sql sqlite:///airfare.db

1. What range of years are represented in the data?

In [28]:
%%sql

SELECT MIN(year), MAX(year), MAX(year) - Min(year) AS 'Year Range'
FROM airfare

 * sqlite:///airfare.db
Done.


MIN(year),MAX(year),Year Range
1996,2018,22


2. What are the shortest and longest-distanced flights, and between which 2 cities are they? 

In [29]:
%%sql

WITH 
far AS (SELECT * FROM airfare WHERE nsmiles IN (SELECT MAX(nsmiles) FROM airfare) LIMIT 1),
near AS (SELECT * FROM airfare WHERE nsmiles IN (SELECT MIN(nsmiles) FROM airfare) LIMIT 1)
SELECT * FROM far
UNION
SELECT * FROM near

 * sqlite:///airfare.db
Done.


Year,quarter,citymarketid_1,citymarketid_2,city1,city2,nsmiles,passengers,fare,carrier_lg,large_ms,fare_lg,carrier_low,lf_ms,fare_low,table_1_flag,Geocoded_City1,Geocoded_City2
2001,1,32575,33570,"Los Angeles, CA (Metropolitan Area)","San Diego, CA",109,210,126.88,AA,0.66,125.13,UA,0.3,122.98,1,"Los Angeles, CA (Metropolitan Area)(34.052238, -118.243344)","San Diego, CA(32.71576, -117.163817)"
2002,2,32467,30559,"Miami, FL (Metropolitan Area)","Seattle, WA",2724,472,210.25,AA,0.25,221.65,CO,0.12,192.82,1,"Miami, FL (Metropolitan Area)(44.977479, -93.264346)","Seattle, WA(47.603229, -122.33028)"


1. How many distinct cities are represented in the data (regardless of whether it is the source or destination)?

In [30]:
%%sql

WITH cities AS (
    SELECT citymarketid_1 FROM airfare
    UNION    
    SELECT citymarketid_2 FROM airfare
)
SELECT COUNT(citymarketid_1) FROM cities

 * sqlite:///airfare.db
Done.


COUNT(citymarketid_1)
163


4. Which airline appear most frequently as the carrier with the lowest fare (ie. carrier_low)? How about the airline with the largest market share (ie. carrier_lg)?

In [42]:
%%sql

SELECT carrier_low, COUNT(carrier_low) FROM airfare
WHERE carrier_low IS NOT NULL
GROUP BY carrier_low
ORDER BY 2 ASC
LIMIT 5

 * sqlite:///airfare.db
Done.


carrier_low,COUNT(carrier_low)
E9,1
A7,2
OO,2
EV,3
N5,3


In [43]:
%%sql

SELECT carrier_lg, COUNT(carrier_lg) FROM airfare
WHERE carrier_lg IS NOT NULL
GROUP BY carrier_lg
ORDER BY 2 DESC
LIMIT 2

 * sqlite:///airfare.db
Done.


carrier_lg,COUNT(carrier_lg)
WN,23659
DL,15789


5. How many instances are there where the carrier with the largest market share is not the carrier with the lowest fare? What is the average difference in fare?

In [48]:
%%sql

SELECT COUNT(*) AS 'Routes' FROM airfare
WHERE carrier_lg != carrier_low

 * sqlite:///airfare.db
Done.


Routes
59851


In [51]:
%%sql

SELECT ROUND(AVG(fare_lg - fare_low), 0) AS 'Avg Fare Diff' FROM airfare

 * sqlite:///airfare.db
Done.


Avg Fare Diff
33.0


6. What is the percent change in average fare from 2007 to 2017 by flight? How about from 1997 to 2017? 

In [75]:
%%sql

WITH
fare1997 AS (SELECT year, city1, city2, AVG(fare) AS 'avgf' FROM airfare GROUP BY year, city1, city2 HAVING year = 1997),
fare2007 AS (SELECT city1, city2, AVG(fare) AS 'avgf' FROM airfare GROUP BY year, city1, city2 HAVING year = 2007),
fare2017 AS (SELECT city1, city2, AVG(fare) AS 'avgf' FROM airfare GROUP BY year, city1, city2 HAVING year = 2017),
q97_07  AS (SELECT fare1997.year, fare1997.city1, fare1997.city2, fare1997.avgf AS 'F97', fare2007.avgf AS 'F07' FROM fare1997 JOIN fare2007 ON fare1997.city1 = fare2007.city1 AND fare1997.city2 = fare2007.city2),
q97_07_17 AS (SELECT q97_07.year, q97_07.city1, q97_07.city2, q97_07.F97 AS 'F97', q97_07.F07 AS 'F07', fare2017.avgf AS 'F17' FROM q97_07 JOIN fare2017 ON q97_07.city1 = fare2017.city1 AND q97_07.city2 = fare2017.city2)
SELECT city1, city2, 
ROUND((f07 - f97)/ f97 * 100, 2) AS 'Avg Fare % 97 to 07',
ROUND((f17 - f97)/ f97 * 100, 2) AS 'Avg Fare % 97 to 17'
FROM q97_07_17
LIMIT 10


 * sqlite:///airfare.db
Done.


city1,city2,Avg Fare % 97 to 07,Avg Fare % 97 to 17
"Albany, NY","Chicago, IL",-33.5,-3.76
"Albany, NY","Los Angeles, CA (Metropolitan Area)",-22.44,1.89
"Albany, NY","Miami, FL (Metropolitan Area)",25.87,12.04
"Albany, NY","Orlando, FL",-0.45,27.5
"Albany, NY","Tampa, FL (Metropolitan Area)",-4.14,39.83
"Albany, NY","Washington, DC (Metropolitan Area)",-40.61,-10.7
"Albuquerque, NM","Atlanta, GA (Metropolitan Area)",46.23,41.82
"Albuquerque, NM","Austin, TX",33.31,53.87
"Albuquerque, NM","Boston, MA (Metropolitan Area)",10.91,21.26
"Albuquerque, NM","Chicago, IL",19.14,53.65
